Zhang Chao, 2025.05.11<br>
Plotting the grid-scale contribution coefficient

In [1]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gpd
import pymannkendall as mk
import statsmodels.api as sm
import matplotlib.patches as patches
import sys
import matplotlib as mpl
import os
import scipy
from matplotlib.colors import Normalize
from cartopy.feature import ShapelyFeature
import matplotlib.colors as mcolors
sys.path.append('/home/climate/chaoz/code/utils/')
from plot_utils import plot_settings, uneven_cmap


In [2]:
os.chdir('/home/climate/chaoz/project/03Irr_Ts_CN/processed/')
ds = xr.open_dataset('con_coefficient_grid_day.nc')
shp_cn = gpd.read_file('../shapefile/ChinaAll.shp')
shp_nanhai = gpd.read_file('../shapefile/Nanhai.shp')

In [3]:
def plot_bars_area(ax, pos, ds, levels,mycmap, mynorm):
    ax_bar = ax.inset_axes(pos)
    ax_bar.patch.set_alpha(0)
    data = np.array(ds.values.flatten())
    data = data[~np.isnan(data)]
    
    f1 = np.histogram(data, bins=levels, density=False)
    colors = [mycmap(mynorm(level)) for level in levels[:-1]]
    ax_bar.bar(np.arange(len(f1[0])), f1[0]/len(data)*100, width=0.8, color=colors, edgecolor='none')
    

    ax_bar.set_xticks([])
    ax_bar.set_yticks([])
    ax_bar.set_ylim([0, 50])
    ax_bar.spines["top"].set_visible(False)
    ax_bar.spines["bottom"].set_visible(True)
    ax_bar.spines["left"].set_visible(False)
    ax_bar.spines["right"].set_visible(False)


def plot_distribution_curve(ax, pos, ds, levels, mycmap, mynorm):
    ax_curve = ax.inset_axes(pos)
    ax_curve.patch.set_alpha(0)
    
    # Flatten and clean data
    data = np.array(ds.values.flatten())
    data = data[~np.isnan(data)]
    
    # Calculate histogram (density=True for PDF)
    hist, bin_edges = np.histogram(data, bins=levels, density=True)
    
    # Calculate bin centers for plotting
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    # Colors for the curve (optional, based on levels)
    colors = [mycmap(mynorm(level)) for level in levels[:-1]]
    
    # Plot probability distribution curve
    ax_curve.plot(bin_centers, hist, color='black', lw=1)  # Plot the curve
    
    # Optional: Fill under the curve with gradient colors
    for i in range(len(hist) - 1):
        ax_curve.fill_between(
            [bin_centers[i], bin_centers[i + 1]],
            [hist[i], hist[i + 1]],
            color=colors[i],
            alpha=0.5,
        )
    ax_curve.spines["top"].set_visible(False)
    ax_curve.spines["bottom"].set_visible(True)
    ax_curve.spines["left"].set_visible(True)
    ax_curve.spines["right"].set_visible(False)
    ax_curve.set_ylabel('Density')


def plot_mean_map(fig, pos, ds, extent,shape_cn,shape_nanhai, mynorm,mycmap, No,title,fameon_sing,nanhai_sign):
    lambert_proj = ccrs.LambertConformal(central_longitude=105, central_latitude=35, standard_parallels=(25, 47))
    ax = fig.add_axes(pos, projection=lambert_proj, frame_on=fameon_sing)
    # Define Lambert Conformal projection (customized for China)

    ds.plot(ax=ax, transform=ccrs.PlateCarree(),#lambert_proj
                    cmap=mycmap, norm=mynorm,add_colorbar=False, rasterized=True) #vmax=vmax, vmin=vmin,
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    # ax1.add_feature(fea_hpa, edgecolor='k', linewidth=0.8)
    fea_cn = ShapelyFeature(shape_cn.geometry, crs=ccrs.PlateCarree())
    fea_nanhai = ShapelyFeature(shape_nanhai.geometry, crs=ccrs.PlateCarree())
    ax.add_feature(fea_cn,facecolor='none',linewidth=.6)
    ax.add_feature(fea_nanhai,facecolor='none',linewidth=.6)
    
    ax.text(0.01, 1.0, No, transform=ax.transAxes, c='k', weight='bold', fontsize=14)
    ax.set_title(title)
    
    plot_distribution_curve(ax, [0.1, 0.05, 0.3, 0.3], ds, np.linspace(0, 1, 20), mycmap, mynorm)
    
    # Add the South China Sea map
    if nanhai_sign:
        hx,vx = 0.0381,0.10
        pos_scs = [ax.get_position().x1-hx,ax.get_position().y0,hx,vx]
        ax_scs = fig.add_axes(pos_scs,projection=lambert_proj,frame_on=True)
        ax_scs.add_feature(fea_cn    ,facecolor='none',linewidth=.6)
        ax_scs.add_feature(fea_nanhai,facecolor='none',linewidth=.6)
        ax_scs.set_extent([107, 120,3,23], crs=ccrs.PlateCarree())
    
    # print(ax_scs.get_position().width,ax_scs.get_position().height)
    return ax


def plot_lat_stats(fig,pos,ds,xlim,ylim):
    ax = fig.add_axes(pos)
    lat_stats = ds.quantile([0.25,0.50,0.75], dim='lon')
    lat = lat_stats.lat
    p25 = lat_stats.sel(quantile=0.25)
    p50 = lat_stats.sel(quantile=0.50)
    p75 = lat_stats.sel(quantile=0.75)
    
    # Fill the region between the 25th and 75th percentiles
    ax.fill_betweenx(lat,p25,p75,color='gray',alpha=0.3)
    # Plot the median (50th percentile) as a solid line
    ax.plot(p50,lat,color='k',linewidth=1.2)
    ax.axvline(0,0,1,color='k',linewidth=1,linestyle='--')
    ax.yaxis.tick_right()
    ax.set_ylim(ylim)
    ax.set_xlim(xlim)
    # ax.set_xticks([-0.02,0,0.02],[-0.2,0,0.2])
    ax.set_yticks([20,30,40,50],['20N','30N','40N','50N'])
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    
    return ax

In [ ]:
colors1 = plt.cm.BrBG(np.linspace(0.00, 0.40, 128))  # 128* seismic bwr PRGn
colors2 = plt.cm.BrBG(np.linspace(0.60, 1.00, 128))  # 128* seismic
colors = np.vstack((colors1, colors2))
mycmap = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors)
plevel = np.array([0,1,2,4,6,8,10,50])
nlevel = -1 * plevel[1:]
levels1 = np.concatenate([nlevel[::-1], plevel])
cmap1, norm1 = uneven_cmap(levels1, cmap=mycmap)#RdYlBu_r 'BrBG'

mynorm = mpl.colors.Normalize(vmin=0, vmax=1)
mycmap = plt.cm.gist_earth_r #YlGnBu_r

fig = plt.figure(figsize=(8,10))
plot_settings()
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

x0,y0 = 0.01,0.08
xi,yi = 0.155,0.07
hx1,vx1 = 0.35,0.25

pos1 = [x0+(hx1+xi)*0, y0+(vx1+yi)*2, hx1, vx1]
pos2 = [x0+(hx1+xi)*1, y0+(vx1+yi)*2, hx1, vx1]
pos3 = [x0+(hx1+xi)*0, y0+(vx1+yi)*1, hx1, vx1]
pos4 = [x0+(hx1+xi)*1, y0+(vx1+yi)*1, hx1, vx1]
pos5 = [x0+(hx1+xi)*0, y0+(vx1+yi)*0, hx1, vx1]
pos6 = [x0+(hx1+xi)*1, y0+(vx1+yi)*0, hx1, vx1]

extent = [80, 127,15,54]

ax1 = plot_mean_map(fig, pos1, ds.iwu_contrib,  extent,shp_cn,shp_nanhai,mynorm, mycmap, 'a','IWU',False,True)
ax2 = plot_mean_map(fig, pos2, ds.devi_contrib, extent,shp_cn,shp_nanhai,mynorm, mycmap, 'b','$\Delta$EVI',False,True)
ax3 = plot_mean_map(fig, pos3, ds.pr_contrib ,  extent,shp_cn,shp_nanhai,mynorm, mycmap, 'c','Precipitation',False,True)
ax4 = plot_mean_map(fig, pos4, ds.ta_contrib ,  extent,shp_cn,shp_nanhai,mynorm, mycmap, 'd','Temperature',False,True)
ax5 = plot_mean_map(fig, pos5, ds.sr_contrib ,  extent,shp_cn,shp_nanhai,mynorm, mycmap, 'e','Solar radiation',False,True)
ax6 = plot_mean_map(fig, pos6, ds.ws_contrib ,  extent,shp_cn,shp_nanhai,mynorm, mycmap, 'f','Wind speed',False,True)

print(ax3.get_position().width, ax3.get_position().height)

# Plot the latitudinal statistics on the right
pos1_r = [ax1.get_position().x1+0.01,ax1.get_position().y0,0.07,ax1.get_position().height]
pos2_r = [ax2.get_position().x1+0.01,ax2.get_position().y0,0.07,ax2.get_position().height]
pos3_r = [ax3.get_position().x1+0.01,ax3.get_position().y0,0.07,ax3.get_position().height]
pos4_r = [ax4.get_position().x1+0.01,ax4.get_position().y0,0.07,ax4.get_position().height]
pos5_r = [ax5.get_position().x1+0.01,ax5.get_position().y0,0.07,ax5.get_position().height]
pos6_r = [ax6.get_position().x1+0.01,ax6.get_position().y0,0.07,ax6.get_position().height]

ax1_r = plot_lat_stats(fig,pos1_r,ds.iwu_contrib ,[0,0.6],[extent[2],extent[3]])
ax2_r = plot_lat_stats(fig,pos2_r,ds.devi_contrib,[0,0.6],[extent[2],extent[3]])
ax3_r = plot_lat_stats(fig,pos3_r,ds.pr_contrib  ,[0,0.6],[extent[2],extent[3]])
ax4_r = plot_lat_stats(fig,pos4_r,ds.ta_contrib  ,[0,0.6],[extent[2],extent[3]])
ax5_r = plot_lat_stats(fig,pos5_r,ds.sr_contrib  ,[0,0.6],[extent[2],extent[3]])
ax6_r = plot_lat_stats(fig,pos6_r,ds.ws_contrib  ,[0,0.6],[extent[2],extent[3]])

hx = 0.17
pos01 = [ax1.get_position().x0+0.05            ,ax1.get_position().y1+0.07,hx,0.13]
pos02 = [ax1.get_position().x0+0.05+(hx+0.05)*1,ax1.get_position().y1+0.07,hx,0.13]
pos03 = [ax1.get_position().x0+0.05+(hx+0.05)*2,ax1.get_position().y1+0.07,hx,0.13]
pos04 = [ax1.get_position().x0+0.05+(hx+0.05)*3,ax1.get_position().y1+0.07,hx,0.13]


pos_cbar1 = [0.1,0.04,0.8,0.01]

ax_cb1 = fig.add_axes(pos_cbar1)

mpl.colorbar.ColorbarBase(ax=ax_cb1, cmap=mycmap, norm=mynorm, orientation='horizontal')
ax_cb1.set_xlabel('Contribution coefficient',labelpad=0)

plt.savefig('../figures/Figure_S10.png',dpi=300)